In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers
def preprocess_imagecont(image, label):
  image = tf.image.resize(image, [64,64])
  image = tf.cast(image, tf.float32)
  image = tfa.image.gaussian_filter2d(image, (3,3),3)
  positionsx1 = tf.range(start=0, limit=64, delta=1,dtype=tf.float32)
  positionsy1 = tf.range(start=0, limit=64, delta=1,dtype=tf.float32)
  positionsx1 = tf.expand_dims(tf.tile(tf.expand_dims(positionsx1,0),[64,1]),-1)
  positionsy1 = tf.expand_dims(tf.tile(tf.transpose(tf.expand_dims(positionsy1,0)),[1,64]),-1)
  positions11 = tf.concat([positionsx1,positionsy1],-1)
  u = tf.image.sobel_edges(image[tf.newaxis,:])
  u=tf.squeeze(u,0)
  angle = tf.atan2(u[:,:,0,1],u[:,:,0,0])
  u = tf.reshape(u,[-1,64,64,6])
  u=tf.concat([u,positions11[tf.newaxis,:]],-1)
  u = tf.squeeze(u,0)
  image = tf.concat([image,tf.expand_dims(angle,2),u],-1)
  return image, label

!pip install tensorflow_addons
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import keras.backend as K

unlabeled_dataset_size = 100000
labeled_dataset_size = 5000
image_size = 64
image_channels = 3

# Algorithm hyperparameters
num_epochs = 60
batch_size = 128  # Corresponds to 200 steps per epoch
width = 32
temperature = 1
# Stronger augmentations for contrastive, weaker ones for supervised training
contrastive_augmentation = {"min_area": 0.35, "brightness": 0.6, "jitter": 0.4}
classification_augmentation = {"min_area": 0.75, "brightness": 0.3, "jitter": 0.1}
color_augmentation = {"brightness": 0.3, "jitter": 0.1}
labeled_batch_size = 15

def prepare_dataset():
    # Labeled and unlabeled samples are loaded synchronously
    # with batch sizes selected accordingly
    steps_per_epoch = (unlabeled_dataset_size + labeled_dataset_size) // batch_size
    unlabeled_batch_size = unlabeled_dataset_size // steps_per_epoch
    labeled_batch_size = 128
    print(
        f"batch size is {unlabeled_batch_size} (unlabeled) + {labeled_batch_size} (labeled)"
    )

    unlabeled_train_dataset = (
        tfds.load("stl10", split="unlabelled", as_supervised=True, shuffle_files=True)
        .shuffle(buffer_size=10 * unlabeled_batch_size)
        .map(
          lambda image, label: preprocess_imagecont(image, label))
        .batch(unlabeled_batch_size)
    )
    labeled_train_dataset = (
        tfds.load("stl10", split="train[0:75%]", as_supervised=True, shuffle_files=True)
        .shuffle(buffer_size=10 * labeled_batch_size)
        .map(
          lambda image, label: preprocess_imagecont(image, label))
        .batch(labeled_batch_size)
    )
    labeled_val_dataset = (
        tfds.load("stl10", split="train[75%:95%]", as_supervised=True, shuffle_files=True)
        .shuffle(buffer_size=10 * labeled_batch_size)
        .map(
          lambda image, label: preprocess_imagecont(image, label))
        .batch(labeled_batch_size)
    )
    test_dataset = (
        tfds.load("stl10", split="test", as_supervised=True)
        .map(
          lambda image, label: preprocess_imagecont(image, label))
        .batch(batch_size)
        .prefetch(buffer_size=tf.data.AUTOTUNE)
    )

    # Labeled and unlabeled datasets are zipped together
    train_dataset = tf.data.Dataset.zip(
        (unlabeled_train_dataset, labeled_train_dataset)
    ).prefetch(buffer_size=tf.data.AUTOTUNE)

    return train_dataset, labeled_train_dataset, test_dataset, labeled_val_dataset

# Load STL10 dataset
train_dataset, labeled_train_dataset, test_dataset, labeled_val_dataset = prepare_dataset()

es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min',patience=2)
from keras.callbacks import ModelCheckpoint
filepath = 'my_best_model.hdf5'
checkpoint1 = ModelCheckpoint(filepath='ch',save_format=tf,monitor='val_loss',
                            save_weights_only=True,
                             verbose=1, 
                             save_best_only=True,
                             mode='min')
checkpoint2 = ModelCheckpoint(filepath='ch2',save_format=tf,monitor='val_sparse_categorical_accuracy',
                            save_weights_only=True,
                             verbose=1, 
                             save_best_only=True,
                             mode='min')
checkpoint3 = ModelCheckpoint(filepath='ch3',save_format=tf,monitor='val_loss',
                            save_weights_only=True,
                             verbose=1, 
                             save_best_only=True,
                             mode='min')
checkpoint4= ModelCheckpoint(filepath='ch4',save_format=tf,monitor='val_loss',
                            save_weights_only=True,
                             verbose=1, 
                             save_best_only=True,
                             mode='min')
checkpointR = ModelCheckpoint(filepath='chr',save_format=tf,monitor='val_loss',
                            save_weights_only=True,
                             verbose=1, 
                             save_best_only=True,
                             mode='min')
checkpointR2 = ModelCheckpoint(filepath='chr2',save_format=tf,monitor='val_loss',
                            save_weights_only=True,
                             verbose=1, 
                             save_best_only=True,
                             mode='min')
checkpointRSmall = ModelCheckpoint(filepath='chrsm',save_format=tf,monitor='val_loss',
                            save_weights_only=True,
                             verbose=1, 
                             save_best_only=True,
                             mode='min')
checkpointL1 = ModelCheckpoint(filepath='chr',save_format=tf,monitor='val_loss',
                            save_weights_only=True,
                             verbose=1, 
                             save_best_only=True,
                             mode='min')
checkpointL2 = ModelCheckpoint(filepath='chr2',save_format=tf,monitor='val_loss',
                            save_weights_only=True,
                             verbose=1, 
                             save_best_only=True,
                             mode='min')
checkpointLRes= ModelCheckpoint(filepath='chrsm',save_format=tf,monitor='val_loss',
                            save_weights_only=True,
                             verbose=1, 
                             save_best_only=True,
                             mode='min')
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min',patience=2)
from keras.callbacks import ModelCheckpoint
filepath = 'my_best_model.hdf5'
checkpoint_stln = ModelCheckpoint(filepath='ch',save_format=tf,monitor='val_loss',
                            save_weights_only=True,
                             verbose=1, 
                             save_best_only=True,
                             mode='min')
checkpoint_stlloc = ModelCheckpoint(filepath='ch2',save_format=tf,monitor='val_loss',
                            save_weights_only=True,
                             verbose=1, 
                             save_best_only=True,
                             mode='min')
def custom_comp(q, k):
    #print(q.shape)
    q1 = tf.reshape(q,[-1,q.shape[1]*q.shape[2],int(q.shape[3]/16),16])
    k1 = tf.reshape(k,[-1,k.shape[1]*k.shape[2],int(k.shape[3]/16),16])
    pairwise_euclidean_distance = tf.reduce_sum(tf.math.square(tf.subtract(tf.transpose(q1[:,:,:,:,tf.newaxis],[0,1,2,3,4],-1) , tf.transpose(k1[:,:,:,:,tf.newaxis],[0,1,2,4,3],-1))),-1)
    pairwise_euclidean_distance = tf.math.l2_normalize(pairwise_euclidean_distance,axis=-1)
    pairwise_euclidean_distance = tf.reshape(pairwise_euclidean_distance,[-1,q.shape[1],q.shape[2],q.shape[3]])
    return pairwise_euclidean_distance

class LocationEncoding(tf.keras.layers.Layer):
  def __init__(self, filters1, filters2):
    super(LocationEncoding, self).__init__()
    self.filters1 = filters1
    self.filters2 = filters2
    #self.positions=positions

    self.conv1 = keras.layers.Conv2D(self.filters1, (1, 1),
                      kernel_initializer='he_normal',
                      name='conv2a',padding="same")
    self.wp = keras.layers.Conv2D(self.filters1, (1, 1),
                      kernel_initializer='he_normal',
                      name='conv2b',padding="same")
    self.conv2 = layers.Conv2D(self.filters2, (3, 3),
                      kernel_initializer='he_normal',
                      name='conv2c',groups=8,padding="same")
    self.conv3 = layers.Conv2D(self.filters2, (3, 3),
                      kernel_initializer='he_normal',
                      name='conv2d',activation='softmax',kernel_regularizer=keras.regularizers.L2(0.0001),padding="same")
    self.conv4 = layers.Conv2D(self.filters2, (1, 1),
                      kernel_initializer='he_normal',
                      name='conv2d',activation='softmax',kernel_regularizer=keras.regularizers.L1(0.0001),padding="same")
    self.conv5 = layers.Conv2D(self.filters2, (1, 1),
                      kernel_initializer='he_normal',
                      name='conv2d',activation='elu',kernel_regularizer=keras.regularizers.L2(0.0001),padding="same")
    self.convl = layers.Conv2D(self.filters2, (1, 1),
                      kernel_initializer='he_normal',
                      name='conv2d',activation='relu',kernel_regularizer=keras.regularizers.L2(0.0001),padding="same")
    
    self.bn = layers.BatchNormalization(name='bn')
    self.bn1 = layers.BatchNormalization(name='bn1')
    self.bn2 = layers.LayerNormalization(name='bn2')
    self.bn3 = layers.LayerNormalization(name='bn3')
    self.bn4 = layers.LayerNormalization(name='bn4')

    #self.dense = tf.keras.layers.Dense(d_model)

  def call(self, input_tensor, image):

    x = self.conv1(input_tensor)
    x = tf.keras.layers.Activation("relu")(x)
    positions = image[:,:,:,-2:]
    positions = tf.tile(positions,[1,1,1,self.filters1])
    positions = layers.AveragePooling2D(pool_size=(image.shape[1]/input_tensor.shape[1],image.shape[2]/input_tensor.shape[2]),strides=(image.shape[1]/input_tensor.shape[1],image.shape[2]/input_tensor.shape[2]))(positions)
    #x1loc = tf.where(tf.reduce_any(x>0),positions,0.0)
    x1 = tf.pad(layers.MaxPooling2D(pool_size=(2,2),padding="same")(x),paddings=[[0,0],[int(x.shape[1]/4),int(x.shape[1]/4)],[int(x.shape[1]/4),int(x.shape[1]/4)],[0,0]])
    x1loc = tf.where(tf.tile(x,[1,1,1,2])>0,positions,0.0)
    #x3 = tf.pad(layers.MaxPooling2D(pool_size=(8,8),padding="same")(x),paddings=[[0,0],[int(x.shape[1]/2),int(x.shape[1]/2)],[int(x.shape[1]/2),int(x.shape[1]/2)],[0,0]])
    x3 = layers.MaxPooling2D(pool_size=(5,5),strides=(1,1),padding="same")(x)
    #x3 = tf.image.resize_with_crop_or_pad(x3,x.shape[1],x.shape[2])
    xa = tf.math.argmax(x,1)
    xa = tf.math.argmax(xa,2)

    x1loc = tf.gather(x1loc,xa,axis=2,batch_dims=1)
    locs = custom_comp(x1loc,x1loc)
    #xpose = self.conv3(locs)
    x = self.bn(x)
    x = layers.Add()([x,self.conv5(x3)])
    #x = self.bn4(x)
    xpose = self.conv4(locs) 
    x = layers.Add()([x,xpose])
    x = self.bn3(x)
    x = self.conv2(x)
    x = self.bn2(x)
    x = tf.keras.layers.Activation("relu")(x)
    return x

def low_level_diff(q, k,positions):
    q = tf.cast(q,dtype=tf.float32)
    k = tf.cast(k,dtype=tf.float32)
    q = tf.reshape(q,[-1,q.shape[1],q.shape[2],int(q.shape[3]/9),9])
    k = tf.reshape(k,[-1,k.shape[1],k.shape[2],int(k.shape[3]/9),9])
    q = tf.transpose(q,[0,1,2,3,4])
    #k = tf.transpose(k,[0,1,2,4,3])
    pairwise = tf.reduce_sum(q[:,:,:,:,:,tf.newaxis] - k[:,:,:,:,tf.newaxis,:],-1)
    pairwise = tf.math.l2_normalize(pairwise,axis=-1)
    pairwise = tf.reshape(pairwise,[-1,q.shape[1],q.shape[2],64])
    #(positions[:,:,tf.newaxis]-positions[:,tf.newaxis,:]),-1)
    return pairwise

data_augmentation = keras.Sequential(
    [
        layers.Normalization(),
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.2),
        layers.RandomZoom(.5, .2)
    ]
)

import cv2
def custom_gabor(shape, dtype=None):
    pi = np.pi
    orientation_spread = np.array([0, pi/4, pi/2, pi*3/4, pi, pi*5/4, pi*3/2, 2*pi])
    scales = np.linspace(2, 3, 6)
    real_kernels = []
    img_kernels = []
#     size, sigma, theta, lambda, gamma aspect ratio
    for orientation in orientation_spread:
        for scale in scales:
            real_kernel = cv2.getGaborKernel((3, 3), 1, orientation, scale, 1, 0)
            imaginary_kernel = cv2.getGaborKernel((3, 3), 1, orientation, scale, 1, np.pi / 2)
            real_kernels.append(real_kernel)
            img_kernels.append(imaginary_kernel)
    stacked_list = np.vstack((real_kernels, img_kernels))
    # stack number equal to number of color channel RGB: ([stacked_list, stacked_list, stacked_list])
    stacked_list = np.array([stacked_list, stacked_list, stacked_list,stacked_list, stacked_list, stacked_list,stacked_list, stacked_list, stacked_list,stacked_list,stacked_list,stacked_list])
    stacked_list = np.einsum('hijk->jkhi', stacked_list)
    print(stacked_list.shape)

    stacked_list = K.variable(stacked_list)
    random = K.random_normal(shape, dtype=dtype)
    return stacked_list

import cv2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 4.1 MB/s 
batch size is 121 (unlabeled) + 128 (labeled)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

...

  0%|          | 0/8000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/stl10/1.0.0.incompleteZ1RMW9/stl10-unlabelled.tfrecord


  0%|          | 0/100000 [00:00<?, ? examples/s]

Dataset stl10 downloaded and prepared to /root/tensorflow_datasets/stl10/1.0.0. Subsequent calls will reuse this data.


In [ ]:
import datetime
from datetime import datetime
import keras.callbacks
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
from sklearn.model_selection import train_test_split
from keras.datasets import cifar10
(train_X, train_y), (test_X, test_y) = cifar10.load_data()
#x_test_new=np.array(x_test_new)
x_train,x_val,y_train,y_val = train_test_split(train_X,train_y,test_size=0.33)

data_augmentation = keras.Sequential(
    [
        layers.Normalization(),
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.2),
        layers.RandomZoom(.5, .2)
    ]
)
data_augmentation.layers[0].adapt(train_X)

170508288/170498071 [==============================] - 11s 0us/step


In [ ]:
checkpoint2 = ModelCheckpoint(filepath='ch2',save_format=tf,monitor='val_sparse_categorical_accuracy',
                            save_weights_only=True,
                             verbose=1, 
                             save_best_only=True,
                             mode='min')

In [ ]:
def NormalModel():
  inputs = layers.Input(shape=(32, 32, 3))
  inputs=data_augmentation(inputs)
  x = layers.Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu")(inputs)
  x1 = layers.Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu")(x)
  x = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(x)
  x = layers.Conv2D(filters=128,kernel_size=(3,3),padding="same", activation="relu")(x)
  x2 = layers.Conv2D(filters=128,kernel_size=(3,3),padding="same", activation="relu")(x)
  x = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(x2)
  x = layers.Conv2D(filters=256,kernel_size=(3,3),padding="same", activation="relu")(x)
  x = layers.Conv2D(filters=256,kernel_size=(3,3),padding="same", activation="relu")(x)
  x = layers.Conv2D(filters=256,kernel_size=(3,3),padding="same", activation="relu")(x)
  x = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(x)
  x = layers.Conv2D(filters=512,kernel_size=(3,3),padding="same", activation="relu")(x)
  x = layers.Conv2D(filters=512,kernel_size=(3,3),padding="same", activation="relu")(x)
  x = layers.Conv2D(filters=512,kernel_size=(3,3),padding="same", activation="relu")(x)
  x = layers.GlobalAveragePooling2D()(x)
  x = layers.Dense(4096,activation='relu')(x)
  x = layers.Dense(4096,activation='relu')(x)
  x = layers.Dense(10,activation='softmax')(x)
  return tf.keras.Model(inputs = inputs,outputs = x)

In [ ]:
def NormalModel2():
  inputs = layers.Input(shape=(64, 64, 12))
  inputs=data_augmentation(inputs)
  x = layers.Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu")(inputs)
  x1 = layers.Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu")(x)
  x1 = layers.BatchNormalization()(x1)
  x = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(x1)
  x = layers.Conv2D(filters=128,kernel_size=(3,3),padding="same", activation="relu")(x1)
  x2 = layers.Conv2D(filters=128,kernel_size=(3,3),padding="same", activation="relu")(x)
  x = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(x2)
  x = layers.BatchNormalization()(x)
  x = layers.Conv2D(filters=256,kernel_size=(3,3),padding="same", activation="relu")(x)
  x = layers.Conv2D(filters=256,kernel_size=(3,3),padding="same", activation="relu")(x)
  x = layers.Conv2D(filters=256,kernel_size=(3,3),padding="same", activation="relu")(x)                                                                             
  x = layers.BatchNormalization()(x)
  x = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(x)
  x = layers.Conv2D(filters=512,kernel_size=(3,3),padding="same", activation="relu")(x)
  x = layers.Conv2D(filters=512,kernel_size=(3,3),padding="same", activation="relu")(x)
  x = layers.Conv2D(filters=512,kernel_size=(3,3),padding="same", activation="relu")(x)
  x = layers.BatchNormalization()(x)
  x = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(x)
  x = layers.Conv2D(filters=512,kernel_size=(3,3),padding="same", activation="relu")(x)
  x = layers.Conv2D(filters=512,kernel_size=(3,3),padding="same", activation="relu")(x)
  x = layers.Conv2D(filters=512,kernel_size=(3,3),padding="same", activation="relu")(x)
  x = layers.BatchNormalization()(x)
  x = layers.GlobalAveragePooling2D()(x)
  x = layers.Dense(4096,activation='relu')(x)
  x = layers.Dense(4096,activation='relu')(x)
  x = layers.Dense(10,activation='softmax')(x)
  return tf.keras.Model(inputs = inputs,outputs = x)

In [ ]:
class BasicBlock(tf.keras.layers.Layer):

    def __init__(self, filter_num, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(filters=filter_num,
                                            kernel_size=(3, 3),
                                            strides=stride,
                                            padding="same")
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.conv2 = tf.keras.layers.Conv2D(filters=filter_num,
                                            kernel_size=(3, 3),
                                            strides=1,
                                            padding="same")
        self.bn2 = tf.keras.layers.BatchNormalization()
        if stride != 1:
            self.downsample = tf.keras.Sequential()
            self.downsample.add(tf.keras.layers.Conv2D(filters=filter_num,
                                                       kernel_size=(1, 1),
                                                       strides=stride))
            self.downsample.add(tf.keras.layers.BatchNormalization())
        else:
            self.downsample = lambda x: x

    def call(self, inputs, training=None, **kwargs):
        residual = self.downsample(inputs)

        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = tf.nn.relu(x)
        x = self.conv2(x)
        x = self.bn2(x, training=training)

        output = tf.nn.relu(tf.keras.layers.add([residual, x]))

        return output
        
def make_basic_block_layer(filter_num, blocks, stride=1):
    res_block = tf.keras.Sequential()
    res_block.add(BasicBlock(filter_num, stride=stride))

    for _ in range(1, blocks):
        res_block.add(BasicBlock(filter_num, stride=1))

    return res_block

In [ ]:
def make_bottleneck_layer(filter_num, blocks, stride=1):
    res_block = tf.keras.Sequential()
    res_block.add(BottleNeck(filter_num, stride=stride))

    for _ in range(1, blocks):
        res_block.add(BottleNeck(filter_num, stride=1))

    return res_block

class BottleNeck(tf.keras.layers.Layer):
    def __init__(self, filter_num, stride=1):
        super(BottleNeck, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(filters=filter_num,
                                            kernel_size=(1, 1),
                                            strides=1,
                                            padding='same')
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.conv2 = tf.keras.layers.Conv2D(filters=filter_num,
                                            kernel_size=(3, 3),
                                            strides=stride,
                                            padding='same')
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.conv3 = tf.keras.layers.Conv2D(filters=filter_num * 4,
                                            kernel_size=(1, 1),
                                            strides=1,
                                            padding='same')
        self.bn3 = tf.keras.layers.BatchNormalization()

        self.downsample = tf.keras.Sequential()
        self.downsample.add(tf.keras.layers.Conv2D(filters=filter_num * 4,
                                                   kernel_size=(1, 1),
                                                   strides=stride))
        self.downsample.add(tf.keras.layers.BatchNormalization())

    def call(self, inputs, training=None, **kwargs):
        residual = self.downsample(inputs)

        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = tf.nn.relu(x)
        x = self.conv2(x)
        x = self.bn2(x, training=training)
        x = tf.nn.relu(x)
        x = self.conv3(x)
        x = self.bn3(x, training=training)

        output = tf.nn.relu(tf.keras.layers.add([residual, x]))

        return output

class ResNetTypeII(tf.keras.Model):
    def __init__(self, layer_params):
        super(ResNetTypeII, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(filters=64,
                                            kernel_size=(7, 7),
                                            strides=2,
                                            padding="same")
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.pool1 = tf.keras.layers.MaxPool2D(pool_size=(3, 3),
                                               strides=2,
                                               padding="same")

        self.layer1 = make_bottleneck_layer(filter_num=64,
                                            blocks=layer_params[0])
        self.layer2 = make_bottleneck_layer(filter_num=128,
                                            blocks=layer_params[1],
                                            stride=2)
        self.layer3 = make_bottleneck_layer(filter_num=256,
                                            blocks=layer_params[2],
                                            stride=2)
        self.layer4 = make_bottleneck_layer(filter_num=512,
                                            blocks=layer_params[3],
                                            stride=2)

        self.avgpool = tf.keras.layers.GlobalAveragePooling2D()
        self.fc = tf.keras.layers.Dense(units=10, activation=tf.keras.activations.softmax)
    
    def call(self, inputs, training=None, mask=None):
        x = data_augmentation(inputs)
        x = self.conv1(x)
        x = self.bn1(x, training=training)
        x = tf.nn.relu(x)
        x = self.pool1(x)
        x = self.layer1(x, training=training)
        x = self.layer2(x, training=training)
        x = self.layer3(x, training=training)
        x = self.layer4(x, training=training)
        x = self.avgpool(x)
        output = self.fc(x)

        return output

In [ ]:
class ResNetTypeI(tf.keras.Model):
    def __init__(self, layer_params):
        super(ResNetTypeI, self).__init__()

        self.conv1 = tf.keras.layers.Conv2D(filters=64,
                                            kernel_size=(7, 7),
                                            strides=2,
                                            padding="same")
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.pool1 = tf.keras.layers.MaxPool2D(pool_size=(3, 3),
                                               strides=2,
                                               padding="same")

        self.layer1 = make_basic_block_layer(filter_num=64,
                                             blocks=layer_params[0])
        self.layer2 = make_basic_block_layer(filter_num=128,
                                             blocks=layer_params[1],
                                             stride=2)
        self.layer3 = make_basic_block_layer(filter_num=256,
                                             blocks=layer_params[2],
                                             stride=2)
        self.layer4 = make_basic_block_layer(filter_num=512,
                                             blocks=layer_params[3],
                                             stride=2)

        self.avgpool = tf.keras.layers.GlobalAveragePooling2D()
        self.fc = tf.keras.layers.Dense(units=10, activation=tf.keras.activations.softmax)

    def call(self, inputs, training=None, mask=None):
        x = data_augmentation(inputs)
        x = self.conv1(x)
        x = self.bn1(x, training=training)
        x = tf.nn.relu(x)
        x = self.pool1(x)
        x = self.layer1(x, training=training)
        x = self.layer2(x, training=training)
        x = self.layer3(x, training=training)
        x = self.layer4(x, training=training)
        x = self.avgpool(x)
        output = self.fc(x)

        return output

In [ ]:
def resnet_50():
    return ResNetTypeII(layer_params=[3, 4, 6, 3])

modelRes2=resnet_50()
modelRes2.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss='sparse_categorical_crossentropy',
    metrics='sparse_categorical_accuracy'
)
historyR2 = modelRes2.fit(labeled_train_dataset,validation_data=labeled_val_dataset,epochs=50,callbacks=[checkpointR,tensorboard_callback])

In [ ]:
#modelRes2.evaluate(test_dataset)
modelRes2.save_weights('/content/drive/MyDrive/stlres2')
#modelRes2.evaluate(test_dataset)

In [ ]:
def resnet_101():
    return ResNetTypeII(layer_params=[3, 4, 23, 3])

modelRes3=resnet_101()
modelRes3.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss='sparse_categorical_crossentropy',
    metrics='sparse_categorical_accuracy'
)
historyR3 = modelRes3.fit(labeled_train_dataset,validation_data=labeled_val_dataset,epochs=30,callbacks=[checkpointR2,tensorboard_callback])

In [ ]:
def resnet_34():
    return ResNetTypeI(layer_params=[3, 4, 6, 3])

modelRes=resnet_34()
modelRes.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss='sparse_categorical_crossentropy',
    metrics='sparse_categorical_accuracy'
)
historyR = modelRes.fit(labeled_train_dataset,validation_data=labeled_val_dataset,epochs=50,callbacks=[checkpointRSmall,tensorboard_callback])

In [ ]:
modelRes.load_weights('chr')

In [ ]:
X_test = np.zeros((64,64,64,12))
y_test=np.zeros((64))
for image,lab in test_dataset:
  X_test=np.concatenate([X_test,image])
  y_test=np.concatenate([y_test,lab])
X_test=np.array(X_test)
y_test=np.array(y_test)

In [ ]:
from keras.layers import *
def custom_gabor(shape, dtype=None):
    pi = np.pi
    orientation_spread = np.array([0, pi/4, pi/2, pi*3/4, pi, pi*5/4, pi*3/2, 2*pi])
    scales = np.linspace(2, 3, 6)
    real_kernels = []
    img_kernels = []
#     size, sigma, theta, lambda, gamma aspect ratio
    for orientation in orientation_spread:
        for scale in scales:
            real_kernel = cv2.getGaborKernel((3, 3), 1, orientation, scale, 1, 0)
            imaginary_kernel = cv2.getGaborKernel((3, 3), 1, orientation, scale, 1, np.pi / 2)
            real_kernels.append(real_kernel)
            img_kernels.append(imaginary_kernel)
    stacked_list = np.vstack((real_kernels, img_kernels))
    # stack number equal to number of color channel RGB: ([stacked_list, stacked_list, stacked_list])
    stacked_list = np.array([stacked_list, stacked_list, stacked_list,stacked_list, stacked_list, stacked_list,stacked_list, stacked_list, stacked_list,stacked_list,stacked_list,stacked_list])
    stacked_list = np.einsum('hijk->jkhi', stacked_list)
    print(stacked_list.shape)

    stacked_list = K.variable(stacked_list)
    random = K.random_normal(shape, dtype=dtype)
    return stacked_list

In [ ]:
def LocationModelN():
  inputs = layers.Input(shape=(32, 32, 3))
  x = data_augmentation(inputs)
  #x = layers.Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu")(inputs)
  #x = layers.Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu")(x)
  x = LocationEncoding(256,256)(x,inputs)
  x = LocationEncoding(256,256)(x,inputs)
  #gbr = layers.Conv2D(96, (3, 3), padding='same',kernel_initializer=custom_gabor, data_format='channels_last', input_shape=(64,64,12))(inputs)
  #xgb = layers.Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu")(gbr)
  x = layers.MaxPooling2D(pool_size=(2,2),strides=(2,2))(x)
  x = layers.BatchNormalization()(x)
  x = LocationEncoding(256,256)(x,inputs)
  x = LocationEncoding(256,256)(x,inputs)
  x = LocationEncoding(256,256)(x,inputs)
  x = layers.MaxPooling2D(pool_size=(2,2),strides=(2,2))(x)
  #x = layers.Add()([x,layers.Dense(128)(x1)])
  x = LocationEncoding(800,800)(x,inputs)
  x = LocationEncoding(800,800)(x,inputs)
  x = LocationEncoding(800,800)(x,inputs)
  x = layers.MaxPooling2D(pool_size=(2,2),strides=(2,2))(x)
  #x = layers.Add()([x,layers.Dense(256)(x1)])
  x = LocationEncoding(512,512)(x,inputs)
  x = LocationEncoding(512,512)(x,inputs)
  x = LocationEncoding(512,512)(x,inputs)
  x = layers.GlobalAveragePooling2D()(x)
  x = layers.Dense(4096,activation='relu')(x)
  x = layers.Dense(4096,activation='relu')(x)
  x = layers.Dense(10,activation='softmax')(x)
  return tf.keras.Model(inputs = inputs,outputs = x)

In [ ]:
def LocationModelN2():
  inputs = layers.Input(shape=(32, 32, 3))
  x = data_augmentation(inputs)
  x = layers.Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu")(inputs)
  x = layers.Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu")(x)
  #gbr = layers.Conv2D(96, (3, 3), padding='same',kernel_initializer=custom_gabor, data_format='channels_last', input_shape=(64,64,12))(inputs)
  #xgb = layers.Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu")(gbr)
  x = layers.MaxPooling2D(pool_size=(2,2),strides=(2,2))(x)
  x = layers.BatchNormalization()(x)
  x = LocationEncoding(256,256)(x,inputs)
  x = LocationEncoding(256,256)(x,inputs)
  x = LocationEncoding(256,256)(x,inputs)
  x = layers.MaxPooling2D(pool_size=(2,2),strides=(2,2))(x)
  #x = layers.Add()([x,layers.Dense(128)(x1)])
  x = LocationEncoding(720,720)(x,inputs)
  x = LocationEncoding(720,720)(x,inputs)
  x = LocationEncoding(720,720)(x,inputs)
  x = layers.MaxPooling2D(pool_size=(2,2),strides=(2,2))(x)
  #x = layers.Add()([x,layers.Dense(256)(x1)])
  x = LocationEncoding(512,512)(x,inputs)
  x = LocationEncoding(512,512)(x,inputs)
  x = LocationEncoding(512,512)(x,inputs)
  x = layers.MaxPooling2D(pool_size=(2,2),strides=(2,2))(x)
  x = LocationEncoding(512,512)(x,inputs)
  x = LocationEncoding(512,512)(x,inputs)
  x = LocationEncoding(512,512)(x,inputs)
  x = layers.GlobalAveragePooling2D()(x)
  x = layers.Dense(2048,activation='relu')(x)
  x = layers.Dense(2048,activation='relu')(x)
  x = layers.Dense(10,activation='softmax')(x)
  return tf.keras.Model(inputs = inputs,outputs = x)

In [ ]:
def LocationModel():
  inputs = layers.Input(shape=(64, 64, 12))
  inputs = data_augmentation(inputs)
  x = layers.Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu")(inputs)
  gbr = layers.Conv2D(96, (3, 3), padding='same',kernel_initializer=custom_gabor, data_format='channels_last', input_shape=(64,64,12))(inputs)
  xgb = layers.Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu")(gbr)
  x1 = layers.MaxPooling2D(pool_size=(2,2),strides=(2,2))(layers.Concatenate()([x,xgb]))
  x = layers.BatchNormalization()(x1)
  x = LocationEncoding(128,128)(x,inputs)
  x = LocationEncoding(128,128)(x,inputs)
  x = LocationEncoding(128,128)(x,inputs)
  x = layers.MaxPooling2D(pool_size=(2,2),strides=(2,2))(x)
  #x = layers.Add()([x,layers.Dense(128)(x1)])
  x = LocationEncoding(256,256)(x,inputs)
  x = LocationEncoding(256,256)(x,inputs)
  x = LocationEncoding(256,256)(x,inputs)
  #x = layers.Add()([x,layers.Dense(256)(x1)])
  x = LocationEncoding(512,512)(x,inputs)
  x = LocationEncoding(512,512)(x,inputs)
  x = LocationEncoding(512,512)(x,inputs)
  x = layers.GlobalAveragePooling2D()(x)
  x = layers.Dense(2048,activation='relu')(x)
  x = layers.Dense(2048,activation='relu')(x)
  x = layers.Dense(10,activation='softmax')(x)
  return tf.keras.Model(inputs = inputs,outputs = x)

In [ ]:
def LocationModel2():
  inputs = layers.Input(shape=(64, 64, 12))
  inputs = data_augmentation(inputs)
  x = layers.Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu")(inputs)
  gbr = layers.Conv2D(96, (3, 3), padding='same',kernel_initializer=custom_gabor, data_format='channels_last', input_shape=(64, 64, 12))(inputs)
  xgb = layers.Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu")(gbr)
  #xgb2 = layers.Conv2D(filters=64,kernel_size=(5,5),padding="same", activation="relu")(gbr)
  x1 = layers.MaxPooling2D(pool_size=(2,2),strides=(2,2))(layers.Concatenate()([x,xgb]))
  x = layers.BatchNormalization()(x1)
  x = LocationEncoding(128,128)(x,inputs)
  x = LocationEncoding(128,128)(x,inputs)
  x = LocationEncoding(128,128)(x,inputs)
  x = layers.Add()([x,layers.Dense(128,activation='relu')(x1)])
  x1 = LocationEncoding(256,256)(x,inputs)
  x = LocationEncoding(256,256)(x1,inputs)
  x = LocationEncoding(256,256)(x,inputs)
  x = layers.Add()([x,layers.Dense(256,activation='relu')(x1)])
  x = LocationEncoding(512,512)(x,inputs)
  x = LocationEncoding(512,512)(x,inputs)
  x = LocationEncoding(512,512)(x,inputs)
  x = layers.Add()([x,layers.Dense(512,activation='relu')(x1)])
  x = layers.MaxPooling2D(pool_size=(2,2),strides=(2,2))(x)
  x1 = LocationEncoding(512,512)(x,inputs)
  x = LocationEncoding(512,512)(x1,inputs)
  x = LocationEncoding(512,512)(x,inputs)
  x = layers.Add()([x,layers.Dense(512,activation='relu')(x1)])
  x = layers.GlobalAveragePooling2D()(x)
  x = layers.Dense(2048,activation='relu')(x)
  x = layers.Dense(2048,activation='relu')(x)
  x = layers.Dense(10,activation='softmax')(x)
  return tf.keras.Model(inputs = inputs,outputs = x)

In [ ]:
def locationBlock(inputs,size,inp):
  x1 = LocationEncoding(size,size)(inputs,inp)
  x = LocationEncoding(size,size)(x1,inp)
  x = LocationEncoding(size,size)(x,inp)
  x = layers.Add()([x,layers.Dense(size)(x1)])
  return x

In [ ]:
def LocationModelRes():
  inputs = layers.Input(shape=(64, 64, 12))
  x = data_augmentation(inputs)
  x = layers.Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu")(inputs)
  gbr = layers.Conv2D(96, (3, 3), padding='same',kernel_initializer=custom_gabor, data_format='channels_last', input_shape=(64, 64, 12))(inputs)
  xgb = layers.Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu")(gbr)
  #xgb2 = layers.Conv2D(filters=64,kernel_size=(5,5),padding="same", activation="relu")(gbr)
  x1 = layers.MaxPooling2D(pool_size=(2,2),strides=(2,2))(layers.Concatenate()([x,xgb]))
  x = layers.BatchNormalization()(x1)
  out = locationBlock(x,128,inputs)
  for i in range(50):
    out = locationBlock(out,256,inputs)
  out = locationBlock(out,512,inputs)
  #x = layers.Add()([x,layers.Dense(512)(x1)])
  x = layers.GlobalAveragePooling2D()(x)
  x = layers.Dense(256,activation='relu')(x)
  x = layers.Dense(256,activation='relu')(x)
  x = layers.Dense(10,activation='softmax')(x)
  return tf.keras.Model(inputs = inputs,outputs = x)

In [ ]:
from keras import layers
import keras.backend as K

In [ ]:
loc2 = LocationModel2()
loc2.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss='sparse_categorical_crossentropy',
    metrics='sparse_categorical_accuracy'
)
history = loc2.fit(labeled_train_dataset,validation_data=labeled_val_dataset,epochs=50,callbacks=[checkpoint3,tensorboard_callback])

(3, 3, 12, 96)
Layer LocationEncoding has arguments ['self', 'filters1', 'filters2']
in `__init__` and therefore must override `get_config()`.

Example:

class CustomLayer(keras.layers.Layer):
    def __init__(self, arg1, arg2):
        super().__init__()
        self.arg1 = arg1
        self.arg2 = arg2

    def get_config(self):
        config = super().get_config()
        config.update({
            "arg1": self.arg1,
            "arg2": self.arg2,
        })
        return config


Layer LocationEncoding has arguments ['self', 'filters1', 'filters2']
in `__init__` and therefore must override `get_config()`.

Example:

class CustomLayer(keras.layers.Layer):
    def __init__(self, arg1, arg2):
        super().__init__()
        self.arg1 = arg1
        self.arg2 = arg2

    def get_config(self):
        config = super().get_config()
        config.update({
            "arg1": self.arg1,
            "arg2": self.arg2,
        })
        return config


Epoch 1/50
59/59 [==============================] - ETA: 0s - loss: 10.1304 - sparse_categorical_accuracy: 0.2643
Epoch 1: val_loss improved from inf to 6.83431, saving model to ch3
59/59 [==============================] - 137s 2s/step - loss: 10.1304 - sparse_categorical_accuracy: 0.2643 - val_loss: 6.8343 - val_sparse_categorical_accuracy: 0.0940
Epoch 2/50
59/59 [==============================] - ETA: 0s - loss: 3.0762 - sparse_categorical_accuracy: 0.3227
Epoch 2: val_loss improved from 6.83431 to 5.43288, saving model to ch3
59/59 [==============================] - 104s 2s/step - loss: 3.0762 - sparse_categorical_accuracy: 0.3227 - val_loss: 5.4329 - val_sparse_categorical_accuracy: 0.1080
Epoch 3/50
59/59 [==============================] - ETA: 0s - loss: 1.9422 - sparse_categorical_accuracy: 0.3304
Epoch 3: val_loss improved from 5.43288 to 2.03538, saving model to ch3
59/59 [==============================] - 110s 2s/step - loss: 1.9422 - sparse_categorical_accuracy: 0.3304 - va

In [ ]:
locN = LocationModelN()
locN.load_weights('/content/drive/MyDrive/locN')

(3, 3, 12, 96)


In [ ]:
import tensorflow.keras.layers as layers

In [ ]:
import cv2
locN2 = LocationModelN()
locN2.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss='sparse_categorical_crossentropy',
    metrics='sparse_categorical_accuracy'
)
#locN.load_weights('ch4')
history = locN2.fit(x_train, y_train,validation_data=(x_val,y_val),epochs=50,callbacks=[checkpoint4,tensorboard_callback])

Layer LocationEncoding has arguments ['self', 'filters1', 'filters2']
in `__init__` and therefore must override `get_config()`.

Example:

class CustomLayer(keras.layers.Layer):
    def __init__(self, arg1, arg2):
        super().__init__()
        self.arg1 = arg1
        self.arg2 = arg2

    def get_config(self):
        config = super().get_config()
        config.update({
            "arg1": self.arg1,
            "arg2": self.arg2,
        })
        return config


Layer LocationEncoding has arguments ['self', 'filters1', 'filters2']
in `__init__` and therefore must override `get_config()`.

Example:

class CustomLayer(keras.layers.Layer):
    def __init__(self, arg1, arg2):
        super().__init__()
        self.arg1 = arg1
        self.arg2 = arg2

    def get_config(self):
        config = super().get_config()
        config.update({
            "arg1": self.arg1,
            "arg2": self.arg2,
        })
        return config


Epoch 1/50
1047/1047 [==============================] - ETA: 0s - loss: 3.1107 - sparse_categorical_accuracy: 0.1049
Epoch 1: val_loss did not improve from 1.78970
1047/1047 [==============================] - 175s 161ms/step - loss: 3.1107 - sparse_categorical_accuracy: 0.1049 - val_loss: 3.6258 - val_sparse_categorical_accuracy: 0.0993
Epoch 2/50
1047/1047 [==============================] - ETA: 0s - loss: 2.0123 - sparse_categorical_accuracy: 0.2092
Epoch 2: val_loss did not improve from 1.78970
1047/1047 [==============================] - 171s 163ms/step - loss: 2.0123 - sparse_categorical_accuracy: 0.2092 - val_loss: 2.1131 - val_sparse_categorical_accuracy: 0.1795
Epoch 3/50
1047/1047 [==============================] - ETA: 0s - loss: 1.9185 - sparse_categorical_accuracy: 0.2496
Epoch 3: val_loss did not improve from 1.78970
1047/1047 [==============================] - 171s 163ms/step - loss: 1.9185 - sparse_categorical_accuracy: 0.2496 - val_loss: 20.8686 - val_sparse_categorical

In [ ]:
locN2.load_weights('/content/drive/MyDrive/locationModelN2')

ValueError: ignored

In [ ]:
import cv2
locN2 = LocationModelN()
locN2.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss='sparse_categorical_crossentropy',
    metrics='sparse_categorical_accuracy'
)
#locN.load_weights('ch4')
#history = locN.fit(labeled_train_dataset,validation_data=labeled_val_dataset,epochs=50,callbacks=[checkpoint4,tensorboard_callback])
history = locN2.fit(x_train, y_train,validation_data=(x_val,y_val),epochs=50,callbacks=[checkpoint3,tensorboard_callback])

Layer LocationEncoding has arguments ['self', 'filters1', 'filters2']
in `__init__` and therefore must override `get_config()`.

Example:

class CustomLayer(keras.layers.Layer):
    def __init__(self, arg1, arg2):
        super().__init__()
        self.arg1 = arg1
        self.arg2 = arg2

    def get_config(self):
        config = super().get_config()
        config.update({
            "arg1": self.arg1,
            "arg2": self.arg2,
        })
        return config


Layer LocationEncoding has arguments ['self', 'filters1', 'filters2']
in `__init__` and therefore must override `get_config()`.

Example:

class CustomLayer(keras.layers.Layer):
    def __init__(self, arg1, arg2):
        super().__init__()
        self.arg1 = arg1
        self.arg2 = arg2

    def get_config(self):
        config = super().get_config()
        config.update({
            "arg1": self.arg1,
            "arg2": self.arg2,
        })
        return config


Epoch 1/50
1047/1047 [==============================] - ETA: 0s - loss: 3.2575 - sparse_categorical_accuracy: 0.1000
Epoch 1: val_loss did not improve from 0.99269
1047/1047 [==============================] - 354s 316ms/step - loss: 3.2575 - sparse_categorical_accuracy: 0.1000 - val_loss: 2.4703 - val_sparse_categorical_accuracy: 0.0984
Epoch 2/50
1047/1047 [==============================] - ETA: 0s - loss: 2.2364 - sparse_categorical_accuracy: 0.1796
Epoch 2: val_loss did not improve from 0.99269
1047/1047 [==============================] - 329s 314ms/step - loss: 2.2364 - sparse_categorical_accuracy: 0.1796 - val_loss: 2.7921 - val_sparse_categorical_accuracy: 0.1094
Epoch 3/50
1047/1047 [==============================] - ETA: 0s - loss: 2.0546 - sparse_categorical_accuracy: 0.2377
Epoch 3: val_loss did not improve from 0.99269
1047/1047 [==============================] - 329s 314ms/step - loss: 2.0546 - sparse_categorical_accuracy: 0.2377 - val_loss: 4.7576 - val_sparse_categorical_

In [ ]:
import cv2
locN = LocationModelN()
locN.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss='sparse_categorical_crossentropy',
    metrics='sparse_categorical_accuracy'
)
#locN.load_weights('ch4')
#history = locN.fit(labeled_train_dataset,validation_data=labeled_val_dataset,epochs=50,callbacks=[checkpoint4,tensorboard_callback])
history = locN.fit(x_train, y_train,validation_data=(x_val,y_val),epochs=50,callbacks=[checkpoint4,tensorboard_callback])

Layer LocationEncoding has arguments ['self', 'filters1', 'filters2']
in `__init__` and therefore must override `get_config()`.

Example:

class CustomLayer(keras.layers.Layer):
    def __init__(self, arg1, arg2):
        super().__init__()
        self.arg1 = arg1
        self.arg2 = arg2

    def get_config(self):
        config = super().get_config()
        config.update({
            "arg1": self.arg1,
            "arg2": self.arg2,
        })
        return config


Layer LocationEncoding has arguments ['self', 'filters1', 'filters2']
in `__init__` and therefore must override `get_config()`.

Example:

class CustomLayer(keras.layers.Layer):
    def __init__(self, arg1, arg2):
        super().__init__()
        self.arg1 = arg1
        self.arg2 = arg2

    def get_config(self):
        config = super().get_config()
        config.update({
            "arg1": self.arg1,
            "arg2": self.arg2,
        })
        return config


Epoch 1/50
1047/1047 [==============================] - ETA: 0s - loss: 3.0636 - sparse_categorical_accuracy: 0.1282
Epoch 1: val_loss improved from inf to 2.10808, saving model to ch4
1047/1047 [==============================] - 200s 175ms/step - loss: 3.0636 - sparse_categorical_accuracy: 0.1282 - val_loss: 2.1081 - val_sparse_categorical_accuracy: 0.1974
Epoch 2/50
1047/1047 [==============================] - ETA: 0s - loss: 2.0001 - sparse_categorical_accuracy: 0.2257
Epoch 2: val_loss did not improve from 2.10808
1047/1047 [==============================] - 173s 165ms/step - loss: 2.0001 - sparse_categorical_accuracy: 0.2257 - val_loss: 3.6107 - val_sparse_categorical_accuracy: 0.1004
Epoch 3/50
1047/1047 [==============================] - ETA: 0s - loss: 1.8970 - sparse_categorical_accuracy: 0.2661
Epoch 3: val_loss did not improve from 2.10808
1047/1047 [==============================] - 172s 164ms/step - loss: 1.8970 - sparse_categorical_accuracy: 0.2661 - val_loss: 2.6931 - va

KeyboardInterrupt: ignored

In [ ]:
locN.load_weights('ch4')

In [ ]:
locN.evaluate(test_X,test_y)

313/313 [==============================] - 24s 77ms/step - loss: 1.0779 - sparse_categorical_accuracy: 0.6467


[1.0778511762619019, 0.6467000246047974]

In [ ]:
locN.evaluate(test_dataset)

63/63 [==============================] - 62s 979ms/step - loss: 2.9049 - sparse_categorical_accuracy: 0.5520


[2.904898166656494, 0.5519999861717224]

In [ ]:
locN2.evaluate(test_dataset)

63/63 [==============================] - 72s 969ms/step - loss: 2.2880 - sparse_categorical_accuracy: 0.5339


[2.287968158721924, 0.5338749885559082]

In [ ]:
#n1.evaluate(test_dataset)
normal = NormalModel()
normal.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss='sparse_categorical_crossentropy',
    metrics='sparse_categorical_accuracy'
)
history = normal.fit(x_train, y_train,validation_data=(x_val,y_val),epochs=50,callbacks=[checkpoint3,tensorboard_callback])

Epoch 1/50
1047/1047 [==============================] - ETA: 0s - loss: 2.3414 - sparse_categorical_accuracy: 0.1594
Epoch 1: val_loss improved from inf to 1.86592, saving model to ch3
1047/1047 [==============================] - 19s 17ms/step - loss: 2.3414 - sparse_categorical_accuracy: 0.1594 - val_loss: 1.8659 - val_sparse_categorical_accuracy: 0.2783
Epoch 2/50
1045/1047 [============================>.] - ETA: 0s - loss: 1.7479 - sparse_categorical_accuracy: 0.3254
Epoch 2: val_loss improved from 1.86592 to 1.70393, saving model to ch3
1047/1047 [==============================] - 18s 17ms/step - loss: 1.7479 - sparse_categorical_accuracy: 0.3255 - val_loss: 1.7039 - val_sparse_categorical_accuracy: 0.3442
Epoch 3/50
1045/1047 [============================>.] - ETA: 0s - loss: 1.5477 - sparse_categorical_accuracy: 0.4090
Epoch 3: val_loss improved from 1.70393 to 1.47868, saving model to ch3
1047/1047 [==============================] - 18s 17ms/step - loss: 1.5475 - sparse_categori

In [ ]:
#n1.evaluate(test_dataset)
normal = NormalModel()
normal.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss='sparse_categorical_crossentropy',
    metrics='sparse_categorical_accuracy'
)
history = normal.fit(labeled_train_dataset,validation_data=labeled_val_dataset,epochs=40,callbacks=[checkpoint3,tensorboard_callback])

Epoch 1/40
30/30 [==============================] - ETA: 0s - loss: 9.0117 - sparse_categorical_accuracy: 0.1061
Epoch 1: val_loss improved from inf to 2.30041, saving model to ch3
30/30 [==============================] - 21s 308ms/step - loss: 9.0117 - sparse_categorical_accuracy: 0.1061 - val_loss: 2.3004 - val_sparse_categorical_accuracy: 0.0950
Epoch 2/40
29/30 [============================>.] - ETA: 0s - loss: 2.2973 - sparse_categorical_accuracy: 0.1137
Epoch 2: val_loss did not improve from 2.30041
30/30 [==============================] - 9s 272ms/step - loss: 2.2968 - sparse_categorical_accuracy: 0.1139 - val_loss: 2.4607 - val_sparse_categorical_accuracy: 0.0930
Epoch 3/40
29/30 [============================>.] - ETA: 0s - loss: 2.2925 - sparse_categorical_accuracy: 0.1199
Epoch 3: val_loss did not improve from 2.30041
30/30 [==============================] - 9s 253ms/step - loss: 2.2929 - sparse_categorical_accuracy: 0.1192 - val_loss: 2.3035 - val_sparse_categorical_accuracy

In [ ]:
modeln2 = NormalModel2()
modeln2.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss='sparse_categorical_crossentropy',
    metrics='sparse_categorical_accuracy'
)
#modeln2.load_weights('ch2')
modeln2.fit(labeled_train_dataset,validation_data=labeled_val_dataset,epochs=50,callbacks=[checkpoint2,tensorboard_callback])

Epoch 1/50
30/30 [==============================] - ETA: 0s - loss: 2.1829 - sparse_categorical_accuracy: 0.2155  
Epoch 1: val_loss improved from inf to 39.27564, saving model to ch2
30/30 [==============================] - 2361s 79s/step - loss: 2.1829 - sparse_categorical_accuracy: 0.2155 - val_loss: 39.2756 - val_sparse_categorical_accuracy: 0.0860
Epoch 2/50
24/30 [=======================>......] - ETA: 7:25 - loss: 1.7299 - sparse_categorical_accuracy: 0.3001

KeyboardInterrupt: ignored

In [ ]:
import cv2
locationModel = LocationModel()
locationModel.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss='sparse_categorical_crossentropy',
    metrics='sparse_categorical_accuracy'
)
history = locationModel.fit(labeled_train_dataset,validation_data=labeled_val_dataset,epochs=50,callbacks=[checkpoint1,tensorboard_callback])

In [ ]:
modeln2.evaluate(test_dataset)

63/63 [==============================] - 16s 248ms/step - loss: 3.4762 - sparse_categorical_accuracy: 0.5581


[3.476179838180542, 0.5581250190734863]

In [ ]:
dm4 = LocationModel()
dm4.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss='sparse_categorical_crossentropy',
    metrics='sparse_categorical_accuracy'
)
history = dm4.fit(labeled_train_dataset,validation_data=labeled_val_dataset,epochs=50,callbacks=[checkpoint1,tensorboard_callback])

In [ ]:
dmres = LocationModelRes()
dmres.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss='sparse_categorical_crossentropy',
    metrics='sparse_categorical_accuracy'
)
history = dmres.fit(labeled_train_dataset,validation_data=labeled_val_dataset,epochs=50,callbacks=[checkpointLRes,tensorboard_callback])

In [ ]:
score = dmres.evaluate(test_dataset,verbose=0,batch_size=10)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')
dmres2 = LocationModelRes()
dmres2.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss='sparse_categorical_crossentropy',
    metrics='sparse_categorical_accuracy'
)
dmres2.load_weights('chrsm')
score = dmres2.evaluate(test_dataset,verbose=0,batch_size=10)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Test loss: 1.718310832977295 / Test accuracy: 0.550000011920929
(3, 3, 12, 96)
Test loss: 1.1809693574905396 / Test accuracy: 0.5661249756813049


In [ ]:
score = dmres.evaluate(test_dataset,verbose=0,batch_size=10)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')
dmres2 = LocationModelRes()
dmres2.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss='sparse_categorical_crossentropy',
    metrics='sparse_categorical_accuracy'
)
dmres2.load_weights('chrsm')
score = dmres2.evaluate(test_dataset,verbose=0,batch_size=10)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Test loss: 1.575082540512085 / Test accuracy: 0.5715000033378601
(3, 3, 12, 96)
Test loss: 1.191939353942871 / Test accuracy: 0.562624990940094


In [ ]:
locN.load_weights('ch4')
score = locN.evaluate(test_dataset)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

800/800 [==============================] - 84s 105ms/step - loss: 1.7487 - sparse_categorical_accuracy: 0.5089
Test loss: 1.7487246990203857 / Test accuracy: 0.5088750123977661


In [ ]:
locN.save_weights('/content/drive/MyDrive/locN')

In [ ]:
modeln2.save_weights('/content/drive/MyDrive/n2')

In [ ]:
import cv2
dmres.load_weights('chrsm')
score = dmres.evaluate(test_dataset,verbose=0,batch_size=10)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

In [ ]:
modelRes2.evaluate(test_dataset,verbose=0,batch_size=10)

In [ ]:
modelRes2.load_weights('chr')
modelRes2.evaluate(test_dataset,verbose=0,batch_size=10)

In [ ]:
class AtrousLayer(tf.keras.layers.Layer):
  def __init__(self, filters):
    super(AtrousLayer, self, filters).__init__()
    self.filters = filters
    self.init=tf.keras.initializers.HeNormal()(shape=(3,3,self.filters,self.filters))
    #self.filters1 = tf.Variable(initial_value=self.init,shape=[3,3,256,256],trainable=True)

  def build(self, input_shape):
    self.filters1 = tf.Variable(initial_value=self.init,shape=[3,3,self.filters,self.filters],trainable=True)

  def call(self, inputs):
    x1 = tf.nn.conv2d(inputs,self.filters1,(1,1),padding="SAME")
    x2 = tf.nn.atrous_conv2d(inputs,self.filters1,(1,1),padding="SAME")
    x3 = tf.nn.atrous_conv2d(inputs,self.filters1,(2,2),padding="SAME")
    x4 = tf.nn.atrous_conv2d(inputs,self.filters1,(4,4),padding="SAME")
    x11 = layers.Maximum()([x1,x2,x3,x4])
    #eq = layers.Add()([tf.where(x11==x1,1.0,0.0),tf.where(x11==x2,1.0,0.0),tf.where(x11==x3,1.0,0.0)])
    x4 = x11
    return x11

In [ ]:
labeled_train_dataset_scale = labeled_train_dataset.unbatch().batch(1).map(lambda image,label:scale_img(image,label))

(None, 64, 64, 12)


In [ ]:
def scale_img(image,label):
  BATCH_SIZE = 1
  NUM_BOXES = 1
  IMAGE_HEIGHT = 64
  IMAGE_WIDTH = 64
  CHANNELS = 3
  CROP_SIZE = (64, 64)

  '''return tf.image.crop_and_resize(
    image,
    [[0.25,0.25,0.75,0.75]],
    [0],
    CROP_SIZE,
    method='bilinear',
    extrapolation_value=0.0,
    name=None
  ),'''
  im = tf.pad(tf.image.resize_with_pad(image,32,32,method='bilinear'),[[0,0],[16,16],[16,16],[0,0]])
  print(im.shape)
  return im,label

In [ ]:
score = n.evaluate(labeled_train_dataset_scale,verbose=0,batch_size=10)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')
score = modeln2.evaluate(labeled_train_dataset_scale,verbose=0,batch_size=10)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

In [ ]:
affine(locN,3,3,0,0,0,0)
affine(locN,0,0,0,2,2,0)

tf.Tensor(0.2175, shape=(), dtype=float64)
tf.Tensor(174, shape=(), dtype=int32)


In [ ]:
affine(locN,0,0,0,2,2,0)

tf.Tensor(0.71375, shape=(), dtype=float64)
tf.Tensor(571, shape=(), dtype=int32)


In [ ]:
affine(locN,0,0,0,2,2,0)
affine(locN,2,2,5,2,0,0)

tf.Tensor(0.71375, shape=(), dtype=float64)
tf.Tensor(571, shape=(), dtype=int32)
tf.Tensor(0.655, shape=(), dtype=float64)
tf.Tensor(524, shape=(), dtype=int32)


In [ ]:
#shear/zoom and rotation
affine(locN,0,0,0,0,3,3)
affine(locN,0,0,5,2,0,0)
affine(normal,0,0,0,0,3,3)
affine(normal,0,0,5,2,0,0)

tf.Tensor(0.14875, shape=(), dtype=float64)
tf.Tensor(119, shape=(), dtype=int32)
tf.Tensor(0.71375, shape=(), dtype=float64)
tf.Tensor(571, shape=(), dtype=int32)
tf.Tensor(0.21375, shape=(), dtype=float64)
tf.Tensor(171, shape=(), dtype=int32)
tf.Tensor(0.465, shape=(), dtype=float64)
tf.Tensor(372, shape=(), dtype=int32)


In [ ]:
affine(locN,0,0,0,0,3,3)
affine(locN,0,0,5,2,0,0)
affine(modeln2,0,0,0,0,3,3)
affine(modeln2,0,0,5,2,0,0)

tf.Tensor(0.14875, shape=(), dtype=float64)
tf.Tensor(119, shape=(), dtype=int32)
tf.Tensor(0.71375, shape=(), dtype=float64)
tf.Tensor(571, shape=(), dtype=int32)
tf.Tensor(0.045, shape=(), dtype=float64)
tf.Tensor(36, shape=(), dtype=int32)
tf.Tensor(0.00625, shape=(), dtype=float64)
tf.Tensor(5, shape=(), dtype=int32)


In [ ]:
affine(locN,0,0,3,0,0,0)
#affine(normal,0,0,0,0,3,3)
affine(modeln2,0,0,3,0,0,0)

tf.Tensor(0.1425, shape=(), dtype=float64)
tf.Tensor(114, shape=(), dtype=int32)
tf.Tensor(0.155, shape=(), dtype=float64)
tf.Tensor(124, shape=(), dtype=int32)


In [ ]:
affine(n,0,0,5,0,0,0)
#affine(normal,0,0,0,0,3,3)
affine(n2,0,0,5,0,0,0)

tf.Tensor(0.15873015873015872, shape=(), dtype=float64)
tf.Tensor(10, shape=(), dtype=int32)
tf.Tensor(0.0, shape=(), dtype=float64)
tf.Tensor(0, shape=(), dtype=int32)


In [ ]:
#only shear
affine(locN,0,0,3,0,0,0)
#affine(normal,0,0,0,0,3,3)
affine(normal,0,0,3,0,0,0)

tf.Tensor(0.1425, shape=(), dtype=float64)
tf.Tensor(114, shape=(), dtype=int32)
tf.Tensor(0.19875, shape=(), dtype=float64)
tf.Tensor(159, shape=(), dtype=int32)


In [ ]:
affine_rot(locN,0,0,3,0,0,5)

(8000, 64, 64, 12)
(8000,)
250/250 [==============================] - 65s 253ms/step - loss: 2.6772 - sparse_categorical_accuracy: 0.2724
[2.677175998687744, 0.2723749876022339]
63/63 [==============================] - 61s 955ms/step - loss: 1.6899 - sparse_categorical_accuracy: 0.4974
[1.6899058818817139, 0.4973750114440918]


In [ ]:
affine_rot(locN2,0,0,3,0,0,5)

(8000, 64, 64, 12)
(8000,)
250/250 [==============================] - 69s 259ms/step - loss: 3.8404 - sparse_categorical_accuracy: 0.2719
[3.8403923511505127, 0.2718749940395355]
63/63 [==============================] - 60s 949ms/step - loss: 2.5425 - sparse_categorical_accuracy: 0.4906
[2.54251766204834, 0.4906249940395355]


In [ ]:
affine_rot(locN2,0,0,3,0,0,10)

(8000, 64, 64, 12)
(8000,)
250/250 [==============================] - 68s 270ms/step - loss: 4.0961 - sparse_categorical_accuracy: 0.2709
[4.096139430999756, 0.2708750069141388]
63/63 [==============================] - 60s 949ms/step - loss: 2.5425 - sparse_categorical_accuracy: 0.4906
[2.54251766204834, 0.4906249940395355]


In [ ]:
#affine_translate(locN2,2,2,0,0,0,0)
affine_shear(locN2,2,2,5,0,0,0)

(8000, 64, 64, 12)
250/250 [==============================] - 65s 258ms/step - loss: 4.5111 - sparse_categorical_accuracy: 0.1269
[4.5110859870910645, 0.12687499821186066]
63/63 [==============================] - 61s 956ms/step - loss: 2.2934 - sparse_categorical_accuracy: 0.5340
[2.293421745300293, 0.5339999794960022]


In [ ]:
affine_shear(locN2,2,2,2,0,0,0)

(8000, 64, 64, 12)
250/250 [==============================] - 64s 258ms/step - loss: 4.3710 - sparse_categorical_accuracy: 0.1850
[4.3709797859191895, 0.1850000023841858]
63/63 [==============================] - 61s 955ms/step - loss: 2.2934 - sparse_categorical_accuracy: 0.5340
[2.293421745300293, 0.5339999794960022]


In [ ]:
affine_shear(locN2,2,2,5,0,0,0)

In [ ]:
affine_translate(locN2,10,10,0,0,0,0)
affine_translate(modeln2,10,10,0,0,0,0)

250/250 [==============================] - 65s 258ms/step - loss: 2.4409 - sparse_categorical_accuracy: 0.4949
[2.440918207168579, 0.4948750138282776]
63/63 [==============================] - 61s 955ms/step - loss: 2.2934 - sparse_categorical_accuracy: 0.5340
[2.293421745300293, 0.5339999794960022]
250/250 [==============================] - 6s 24ms/step - loss: 4.2512 - sparse_categorical_accuracy: 0.3801
[4.25121545791626, 0.3801249861717224]
63/63 [==============================] - 13s 207ms/step - loss: 3.3983 - sparse_categorical_accuracy: 0.5190
[3.3983154296875, 0.5189999938011169]


In [ ]:
affine_translate(locN2,2,2,0,0,0,0)
affine_shear(locN2,2,2,0,0,0,0)

250/250 [==============================] - 64s 255ms/step - loss: 2.4902 - sparse_categorical_accuracy: 0.4902
[2.4902303218841553, 0.49024999141693115]
63/63 [==============================] - 60s 948ms/step - loss: 2.5425 - sparse_categorical_accuracy: 0.4906
[2.54251766204834, 0.4906249940395355]


In [ ]:
affine_scale(normal,0,0,2,2,0,0)

(8000, 64, 64, 12)
(8000,)
250/250 [==============================] - 3s 11ms/step - loss: 8.2850 - sparse_categorical_accuracy: 0.1000
[8.284989356994629, 0.10000000149011612]
63/63 [==============================] - 13s 198ms/step - loss: 2.9731 - sparse_categorical_accuracy: 0.4946
[2.9730660915374756, 0.4946250021457672]


In [ ]:
affine_scale(locN2,0,0,2,2,0,0)

(8000, 64, 64, 12)
(8000,)
250/250 [==============================] - 64s 256ms/step - loss: 4.8156 - sparse_categorical_accuracy: 0.1000
[4.8155903816223145, 0.10000000149011612]
63/63 [==============================] - 60s 952ms/step - loss: 2.5425 - sparse_categorical_accuracy: 0.4906
[2.54251766204834, 0.4906249940395355]


In [ ]:
affine_scale(normal,0,0,2,2,0,0)

(8000, 64, 64, 12)
(8000,)
250/250 [==============================] - 3s 12ms/step - loss: 3.6155 - sparse_categorical_accuracy: 0.1000
[3.615548610687256, 0.10000000149011612]
63/63 [==============================] - 14s 217ms/step - loss: 1.8407 - sparse_categorical_accuracy: 0.5136
[1.8406566381454468, 0.5136250257492065]


In [ ]:
H

NameError: ignored

In [ ]:
normal.save_weights('/content/drive/MyDrive/normal')

In [ ]:
affine_translate(locN,2,2,0,0,0,0)

250/250 [==============================] - 63s 254ms/step - loss: 1.7046 - sparse_categorical_accuracy: 0.4961
[1.7045879364013672, 0.4961250126361847]
63/63 [==============================] - 61s 964ms/step - loss: 1.6899 - sparse_categorical_accuracy: 0.4974
[1.6899058818817139, 0.4973750114440918]


In [ ]:
affine_rot(modeln2,0,0,3,0,0,5)

(8000, 64, 64, 12)
(8000,)
250/250 [==============================] - 6s 23ms/step - loss: 4.5759 - sparse_categorical_accuracy: 0.1646
[4.575926780700684, 0.16462500393390656]
63/63 [==============================] - 13s 197ms/step - loss: 1.7134 - sparse_categorical_accuracy: 0.4235
[1.7134194374084473, 0.4235000014305115]


In [ ]:
affine_translate(modeln2,2,2,0,0,0,0)

250/250 [==============================] - 6s 22ms/step - loss: 1.7194 - sparse_categorical_accuracy: 0.4198
[1.7194149494171143, 0.41975000500679016]
63/63 [==============================] - 12s 188ms/step - loss: 1.7134 - sparse_categorical_accuracy: 0.4235
[1.7134194374084473, 0.4235000014305115]


In [ ]:
#only zoom
affine(locN,0,0,0,2,0,0)
#affine(normal,0,0,0,0,3,3)
affine(normal,0,0,0,2,0,0)

tf.Tensor(0.71375, shape=(), dtype=float64)
tf.Tensor(571, shape=(), dtype=int32)
tf.Tensor(0.465, shape=(), dtype=float64)
tf.Tensor(372, shape=(), dtype=int32)


In [ ]:
affine(modeln2,0,0,0,2,0,0)

2/2 [==============================] - 0s 7ms/step - loss: 0.0000e+00 - sparse_categorical_accuracy: 0.0000e+00
[0.0, 0.0]
63/63 [==============================] - 12s 188ms/step - loss: 2.8068 - sparse_categorical_accuracy: 0.5031
[2.806838274002075, 0.503125011920929]


In [ ]:
affine(locN2,0,0,0,2,0,0)

In [ ]:
affine(locN,0,0,0,3,3,0)
affine(normal,0,0,0,3,3,0)

tf.Tensor(0.0, shape=(), dtype=float64)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(0.91375, shape=(), dtype=float64)
tf.Tensor(731, shape=(), dtype=int32)


In [ ]:
score = locN.evaluate(labeled_train_dataset_scale,verbose=0,batch_size=10)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')
score = normal.evaluate(labeled_train_dataset_scale,verbose=0,batch_size=10)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Test loss: 3.482125759124756 / Test accuracy: 0.17679999768733978
Test loss: 5.611385345458984 / Test accuracy: 0.19733333587646484


In [ ]:
score = dmres.evaluate(labeled_train_dataset_scale,verbose=0,batch_size=10)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Test loss: 9.769747734069824 / Test accuracy: 0.15493333339691162


In [ ]:
score = n.evaluate(labeled_train_dataset_scale,verbose=0,batch_size=10)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')
score = modeln2.evaluate(labeled_train_dataset_scale,verbose=0,batch_size=10)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

In [ ]:
score = dmres.evaluate(labeled_train_dataset_scale,verbose=0,batch_size=10)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')
score = dm3.evaluate(labeled_train_dataset_scale,verbose=0,batch_size=10)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')
score = dm4.evaluate(labeled_train_dataset_scale,verbose=0,batch_size=10)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')
score = n.evaluate(labeled_train_dataset_scale,verbose=0,batch_size=10)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

In [ ]:
score = modeln2.evaluate(test_dataset,verbose=0,batch_size=10)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')
score = n.evaluate(test_dataset,verbose=0,batch_size=10)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')
score = dm41.evaluate(test_dataset,verbose=0,batch_size=10)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')
score = modelRes.evaluate(test_dataset,verbose=0,batch_size=10)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

In [ ]:
affine(dmres,1,0,3,0,0,0)

tf.Tensor(0.0, shape=(), dtype=float64)
tf.Tensor(0, shape=(), dtype=int32)


In [ ]:
affine(modelRes)
affine(dm41)
affine(n)
affine(modeln2)

In [ ]:
def affine_translate(model,tx,ty,shear,zx,zy,theta):
    total_acc=0
    n=0
    predictions1 = model.predict(test_dataset)
    image2 = []
    labels = []
    #image2 =np.array(image2)
    for image,lab in test_dataset.unbatch(): 
      tr = tfa.image.translate(image,[tx,ty])
      #image2=np.concatenate((image2,tr),axis=0)
      image2.append(tr)
      labels.append(lab)
    #image2 = image2[1:,:,:,:]
    image2 = tf.stack(image2,0)
    labels = np.array(labels)
    n+=image2.shape[0]
    predictions2 = model.predict(image2)
    accuracy = tf.reduce_sum(tf.cast(tf.argmax(predictions2, axis=-1) == tf.argmax(predictions1, axis=-1),dtype=tf.int32))
    total_acc+=accuracy
    print(model.evaluate(image2,labels))
    print(model.evaluate(test_dataset))

In [ ]:
def affine_shear(model,tx,ty,shear,zx,zy,theta):
    total_acc=0
    n=0
    predictions1 = model.predict(test_dataset)
    image2 = []
    labels = []
    #image2 =np.array(image2)
    for image,lab in test_dataset.unbatch(): 
      tr = tfa.image.transform(image,(1,shear,0,0,1,0,0,0))
      image2.append(tr)
      labels.append(lab)
    #image2 = image2[1:,:,:,:]
    image2 = tf.stack(image2,0)
    print(image2.shape)
    labels = np.array(labels)
    n+=image2.shape[0]
    predictions2 = model.predict(image2)
    accuracy = tf.reduce_sum(tf.cast(tf.argmax(predictions2, axis=-1) == tf.argmax(predictions1, axis=-1),dtype=tf.int32))
    total_acc+=accuracy
    print(model.evaluate(image2,labels))
    print(model.evaluate(test_dataset))

In [ ]:
def affine_rot(model,tx,ty,shear,zx,zy,theta):
    total_acc=0
    n=0
    predictions1 = model.predict(test_dataset)
    image2 = []
    labels = []
    #image2 =np.array(image2)
    for image,lab in test_dataset.unbatch(): 
      tr = tfa.image.rotate(image,theta)
      image2.append(tr)
      labels.append(lab)
    #image2 = image2[1:,:,:,:]
    labels = np.array(labels)
    image2 = tf.stack(image2,0)
    print(image2.shape)
    print(labels.shape)
    n+=image2.shape[0]
    predictions2 = model.predict(image2)
    accuracy = tf.reduce_sum(tf.cast(tf.argmax(predictions2, axis=-1) == tf.argmax(predictions1, axis=-1),dtype=tf.int32))
    total_acc+=accuracy
    print(model.evaluate(image2,labels))
    print(model.evaluate(test_dataset))

In [ ]:
def affine_scale(model,tx,ty,shear,zx,zy,theta):
    total_acc=0
    n=0
    predictions1 = model.predict(test_dataset)
    image2 = []
    labels = []
    #image2 =np.array(image2)
    for image,lab in test_dataset.unbatch(): 
      tr = tfa.image.transform(image,(zx,0,-2,0,zy,-2,0,0))
      image2.append(tr)
      labels.append(lab)
    #image2 = image2[1:,:,:,:]
    labels = np.array(labels)
    image2 = tf.stack(image2,0)
    print(image2.shape)
    print(labels.shape)
    n+=image2.shape[0]
    predictions2 = model.predict(image2)
    accuracy = tf.reduce_sum(tf.cast(tf.argmax(predictions2, axis=-1) == tf.argmax(predictions1, axis=-1),dtype=tf.int32))
    total_acc+=accuracy
    print(model.evaluate(image2,labels))
    print(model.evaluate(test_dataset))

In [ ]:
print(test_dataset)

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 64, 64, 12), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>


In [ ]:
def affine(model,tx,ty,shear,zx,zy,theta):
    total_acc=0
    n=0
    predictions1 = tf.argmax(model.predict(test_dataset))
    image2 = np.zeros((1,64,64,12))
    image2 =np.array(image2)
    for image,lab in test_dataset:
      for img in image: 
        tr = tf.keras.preprocessing.image.apply_affine_transform(
          img,
          theta=theta,
          tx=tx,
          ty=ty,
          shear=shear,
          zx=zx,
          zy=zy,
          row_axis=1,
          col_axis=2,
          channel_axis=0,
          fill_mode='constant',
          cval=0.0
        )
      image2=np.concatenate((image2,tf.expand_dims(tr,0)),axis=0)
    image2 = image2[1:,:,:,:]
    n+=image2.shape[0]
    predictions2 = model.predict(image2)
    #accuracy = tf.reduce_sum(tf.cast(tf.argmax(predictions2, axis=-1) == tf.argmax(predictions1, axis=-1),dtype=tf.int32))
    #total_acc+=accuracy
    print(model.evaluate(image2))
    print(model.evaluate(test_dataset))
    #print(total_acc/n)
    #print(total_acc)

In [ ]:
def affine(model,tx,ty,shear,zx,zy,theta):
    total_acc=0
    n=0
    predictions1 = tf.argmax(model.predict(test_dataset))
    image2 = np.zeros((1,64,64,12))
    image2 =np.array(image2)
    for image,lab in test_dataset:
      for img in image: 
        tr = tf.keras.preprocessing.image.apply_affine_transform(
          img,
          theta=theta,
          tx=tx,
          ty=ty,
          shear=shear,
          zx=zx,
          zy=zy,
          row_axis=1,
          col_axis=2,
          channel_axis=0,
          fill_mode='constant',
          cval=0.0
        )
      image2=np.concatenate((image2,tf.expand_dims(tr,0)),axis=0)
    image2 = image2[1:,:,:,:]
    n+=image2.shape[0]
    predictions2 = model.predict(image2)
    accuracy = tf.reduce_sum(tf.cast(tf.argmax(predictions2, axis=-1) == tf.argmax(predictions1, axis=-1),dtype=tf.int32))
    total_acc+=accuracy
    score = model.evaluate(test_dataset)
    score2 = model.evaluate(image2)
    print(total_acc/n)
    print(total_acc)
    print(score)
    print(score2)

In [ ]:
affine(dmres,3,3,0,0,0,0)

tf.Tensor(0.04625, shape=(), dtype=float64)
tf.Tensor(37, shape=(), dtype=int32)


In [ ]:
#Scaling
affine(modelRes,0,0,0,2,2,0)
affine(dm41,0,0,0,2,2,0)
affine(n,0,0,0,2,2,0)
affine(modeln2,0,0,0,2,2,0)